In [ ]:
# Author : Raude Killian
# Last modified 16.07.205
# Last Check by Qiaoyue 15.07.2025

In [ ]:
import mne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import warnings
%matplotlib inline

ENCOG_EVENT_DICTIONARY = {
    'congruent_picture_pres': 11,
    'incongruent_picture_pres': 12,

    'left_keypress': 21,
    'right_keypress': 22,

    'onset_fixation_cross': 1,
    'onset_picture_movement': 2,
    'onset_frating_scale': 3,
    'onset_resting_state': 7,
    }

ENCOG_EVENT_RESULTS = {
    'congruent': 80,
    'incongruent': 81,
    }


PARTICIPANTS_FILE = r"L:\Common\Users\Qiaoyue\MEG_project\Data\participants.csv"

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

participants = pd.read_csv(PARTICIPANTS_FILE)  
participants = participants['subID'].astype(str).tolist()
sessions = ["encoding", "recall"]

print(f"✅ {len(participants)} participants list : {participants}")

In [ ]:
# For tests purposes
#participants = ["F103"]
#sessions = ["encoding"]

In [ ]:
# Plotting events (only for visualization)
for subID in participants:

    plots_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\plots\{subID}"
    os.makedirs(plots_folder, exist_ok=True)

    intermediates_folder = rf"C:\Users\killg\Desktop\meg_intermediates\{subID}"
    #intermediates_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\meg_intermediates\{subID}"
    os.makedirs(intermediates_folder, exist_ok=True)

    final_folder = rf"C:\Users\killg\Desktop\finals\{subID}"
    #final_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\finals\{subID}"
    os.makedirs(final_folder, exist_ok=True)

    # Declare filenames 
    processed_meg_filename = rf"{intermediates_folder}/{subID}_{session}_icaed.fif"
    bhv_results_filename = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\bhv\{subID}\Step1\main_{subID}.csv"

    if not (os.path.exists(processed_meg_filename) and os.path.exists(bhv_results_filename)):
        print(f"❌ Missing file(s) for participant {subID}. Skipping...")
        continue
    print(f"Preprocessing: {subID}_recall_session")

    # Open behavioral performance file
    df = pd.read_csv(bhv_results_filename)  
    bhv_results = df['Congruency'].astype(str).tolist()
    bhv_results_ID = [ENCOG_EVENT_RESULTS[i] for i in bhv_results]

    # Open processed meg file and extract events
    processed_meg_file = mne.io.read_raw_fif(processed_meg_filename, preload=True, verbose=False)
    events = mne.find_events(
        processed_meg_file, 
        stim_channel=['STI102'],
        shortest_event=1 / processed_meg_file.info['sfreq'],
        verbose=False
    )

    # Extract events with the onset of picture movement
    events_motion = events[events[:, 2] == 2]
    events_motion[:,2] = bhv_results_ID


    # Plot picture presentation events
    sfreq = processed_meg_file.info['sfreq'] 
    fig = mne.viz.plot_events(events, event_id=ENCOG_EVENT_DICTIONARY, show=False, on_missing="ignore")
    ax = fig.axes[0]
    xticks = ax.get_xticks()
    xtick_labels_minutes = [f"{(x / sfreq) / 60:.0f}" for x in xticks]

    ax.set_xticks(xticks)
    ax.set_xticklabels(xtick_labels_minutes)
    ax.set_xlabel("Time (minutes)")

    fig.suptitle("Encoding Events", fontsize=14)

    fig.set_size_inches(12, 6)
    fig.savefig(f"{plots_folder}/{subID}_events_encoding_plots.png")
    plt.close(fig) 

    # Plot full events
    fig = mne.viz.plot_events(events_motion, event_id=ENCOG_EVENT_RESULTS, show=False, on_missing="ignore")
    ax = fig.axes[0]
    xticks = ax.get_xticks()
    xtick_labels_minutes = [f"{(x / sfreq) / 60:.0f}" for x in xticks]

    ax.set_xticks(xticks)
    ax.set_xticklabels(xtick_labels_minutes)
    ax.set_xlabel("Time (minutes)")

    fig.suptitle("All Events", fontsize=14)

    fig.set_size_inches(12, 6)
    fig.savefig(f"{plots_folder}/{subID}_events_full_plots.png")
    plt.close(fig)

    print(f"✅ Participant: {subID} sucessfully completed !  \n")

In [ ]:
# Extract events and save epochs/evoked
numbers_congruent_trials = []
numbers_incongruent_trials = []
for subID in participants:

    plots_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\plots\{subID}"
    os.makedirs(plots_folder, exist_ok=True)

    intermediates_folder = rf"C:\Users\killg\Desktop\meg_intermediates\{subID}"
    #intermediates_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\meg_intermediates\{subID}"
    os.makedirs(intermediates_folder, exist_ok=True)

    final_folder = rf"C:\Users\killg\Desktop\finals\{subID}"
    #final_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\finals\{subID}"
    os.makedirs(final_folder, exist_ok=True)

    # Declare filenames 
    processed_meg_filename = rf"{intermediates_folder}/{subID}_{session}_icaed.fif"
    bhv_results_filename = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\bhv\{subID}\Step1\main_{subID}.csv"

    if not (os.path.exists(processed_meg_filename) and os.path.exists(bhv_results_filename)):
        print(f"❌ Missing file(s) for participant {subID}. Skipping...")
        continue
    print(f"Preprocessing: {subID}_recall_session")

    # Open behavioral performance file
    df = pd.read_csv(bhv_results_filename)  
    bhv_results = df['Congruency'].astype(str).tolist()
    bhv_results_ID = [ENCOG_EVENT_RESULTS[i] for i in bhv_results]

    # Open processed meg file and extract events
    processed_meg_file = mne.io.read_raw_fif(processed_meg_filename, preload=True, verbose=False)
    events = mne.find_events(
        processed_meg_file, 
        stim_channel=['STI102'],
        shortest_event=1 / processed_meg_file.info['sfreq'],
        verbose=False
    )

    # Extract events with the onset of picture movement
    events_motion = events[events[:, 2] == 2]
    events_motion[:,2] = bhv_results_ID

    epochs = mne.Epochs(
        processed_meg_file, 
        events=events_motion,
        event_id = ENCOG_EVENT_RESULTS,   
        tmin=-0.5,           
        tmax=2,            
        baseline=None,  
        preload=True,
        verbose=False,
        on_missing='ignore'       
    )

    count_congruent_trials = len(epochs['congruent'])
    count_incongruent_trials = len(epochs['incongruent'])

    numbers_congruent_trials.append(count_congruent_trials)
    numbers_incongruent_trials.append(count_incongruent_trials)

    print(f"Number of correct-congruent trials: {count_congruent_trials}")
    print(f"Number of correct-incongruent trials: {count_incongruent_trials}")

    epochs['congruent'].save(rf"{final_folder}/{subID}_epochs_encod_congruent.fif", overwrite=True, verbose=False)
    epochs['incongruent'].save(rf"{final_folder}/{subID}_epochs_encod_incongruent.fif", overwrite=True, verbose=False)

    # Average trials, interpolate bads, apply baseline correction in this order
    evoked_cong = epochs['congruent'].copy().average(method='mean')
    evoked_cong.interpolate_bads(reset_bads=True, verbose = False)
    evoked_cong.apply_baseline((-0.2,0),verbose = False)
    evoked_cong.save(rf"{final_folder}/{subID}_encod_cong_evoked.fif", overwrite=True, verbose=False)

    evoked_incong = epochs['incongruent'].copy().average(method='mean')
    evoked_incong.interpolate_bads(reset_bads=True, verbose = False)
    evoked_incong.apply_baseline((-0.2,0),verbose = False)
    evoked_incong.save(rf"{final_folder}/{subID}_encod_incong_evoked.fif", overwrite=True, verbose=False)


    print(f"✅ Participant: {subID} sucessfully completed !  \n")

print(f"Average number of congruent trials: {np.mean(numbers_congruent_trials):.2f} ± {np.std(numbers_congruent_trials):.2f}")
print(f"Average number of incongruent trials: {np.mean(numbers_incongruent_trials):.2f} ± {np.std(numbers_incongruent_trials):.2f}")